In [1]:
!nvidia-smi

Tue Oct 18 22:16:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls -alh /mydrive/yolov3_gumby_n_friends 

Mounted at /content/gdrive
total 739M
-rw------- 1 root root 269M Oct 15 23:37 gnf_images.zip
-rw------- 1 root root 8.2K Oct 16 14:26 yolov3_FIVE_categories.cfg
-rw------- 1 root root 235M Oct 18 21:37 yolov3_training_last.weights
-rw------- 1 root root 235M Oct 18 22:14 yolov3_training_restart0.weights


In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15494, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15494 (delta 0), reused 1 (delta 0), pack-reused 15490
Receiving objects: 100% (15494/15494), 14.09 MiB | 20.40 MiB/s, done.
Resolving deltas: 100% (10411/10411), done.


In [4]:
%cd darknet

/content/darknet


In [5]:
!ls -alh .

total 272K
drwxr-xr-x 14 root root 4.0K Oct 18 22:17 .
drwxr-xr-x  1 root root 4.0K Oct 18 22:17 ..
drwxr-xr-x  4 root root 4.0K Oct 18 22:17 3rdparty
drwxr-xr-x  3 root root 4.0K Oct 18 22:17 build
-rwxr-xr-x  1 root root  35K Oct 18 22:17 build.ps1
drwxr-xr-x  3 root root 4.0K Oct 18 22:17 cfg
drwxr-xr-x  2 root root 4.0K Oct 18 22:17 .circleci
drwxr-xr-x  3 root root 4.0K Oct 18 22:17 cmake
-rw-r--r--  1 root root  25K Oct 18 22:17 CMakeLists.txt
-rw-r--r--  1 root root 1.4K Oct 18 22:17 DarknetConfig.cmake.in
-rw-r--r--  1 root root 9.4K Oct 18 22:17 darknet_images.py
-rw-r--r--  1 root root  11K Oct 18 22:17 darknet.py
-rw-r--r--  1 root root 6.6K Oct 18 22:17 darknet_video.py
drwxr-xr-x  3 root root 4.0K Oct 18 22:17 data
drwxr-xr-x  8 root root 4.0K Oct 18 22:17 .git
drwxr-xr-x  4 root root 4.0K Oct 18 22:17 .github
-rw-r--r--  1 root root  686 Oct 18 22:17 .gitignore
-rwxr-xr-x  1 root root  110 Oct 18 22:17 image_yolov3.sh
-rwxr-xr-x  1 root root  110 Oct 18 22:17 image_yolov4

In [6]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [7]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
# 5 classes, filters = (num_classes+5)*3 = 30
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 10000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=30@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=30@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=30@' cfg/yolov3_training.cfg

In [9]:
!echo "Gumby" > data/obj.names
!echo "Pokey" >> data/obj.names
!echo "Minga" >> data/obj.names
!echo "Goo" >> data/obj.names
!echo "Prickle" >> data/obj.names
!cat data/obj.names

Gumby
Pokey
Minga
Goo
Prickle


In [10]:
!echo -e 'classes= 5\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3_gumby_n_friends' > data/obj.data
!mkdir data/obj
!cat data/obj.data

classes= 5
train  = data/train.txt
valid  = data/test.txt
names = data/obj.names
backup = /mydrive/yolov3_gumby_n_friends


In [11]:
!unzip /mydrive/yolov3_gumby_n_friends/gnf_images.zip -d data/obj

Archive:  /mydrive/yolov3_gumby_n_friends/gnf_images.zip
  inflating: data/obj/0_Color.jpeg   
  inflating: data/obj/0_Color.txt    
  inflating: data/obj/10_Color.jpeg  
  inflating: data/obj/10_Color.txt   
  inflating: data/obj/11_Color.jpeg  
  inflating: data/obj/11_Color.txt   
  inflating: data/obj/12_Color.jpeg  
  inflating: data/obj/12_Color.txt   
  inflating: data/obj/13_Color.jpeg  
  inflating: data/obj/13_Color.txt   
  inflating: data/obj/14_Color.jpeg  
  inflating: data/obj/14_Color.txt   
  inflating: data/obj/15641_1.jpg    
  inflating: data/obj/15641_1.txt    
  inflating: data/obj/15_Color.jpeg  
  inflating: data/obj/15_Color.txt   
  inflating: data/obj/16_Color.jpeg  
  inflating: data/obj/16_Color.txt   
  inflating: data/obj/17_Color.jpeg  
  inflating: data/obj/17_Color.txt   
  inflating: data/obj/18_Color.jpeg  
  inflating: data/obj/18_Color.txt   
  inflating: data/obj/19_Color.jpeg  
  inflating: data/obj/19_Color.txt   
  inflating: data/obj/1_Color.j

In [12]:
import glob
images_list = glob.glob("data/obj/*.j*")
print(images_list)
len(images_list)

['data/obj/gnf24_Color.jpeg', 'data/obj/PXL_20221008_205808872.jpg', 'data/obj/PXL_20221008_212634155.jpg', 'data/obj/y6_Color.jpg', 'data/obj/g22_Color.jpg', 'data/obj/y20_Color.jpg', 'data/obj/l_Color.jpeg', 'data/obj/gnf21_Color.jpeg', 'data/obj/g20_Color.jpg', 'data/obj/PXL_20221008_205819519.jpg', 'data/obj/gnf22_Color.jpeg', 'data/obj/gnf63_Color.jpeg', 'data/obj/PXL_20221012_002842122.jpg', 'data/obj/18_Color.jpeg', 'data/obj/gnf37_Color.jpeg', 'data/obj/b_Color.jpeg', 'data/obj/15641_1.jpg', 'data/obj/16_Color.jpeg', 'data/obj/g7_Color.jpg', 'data/obj/IMG_20221010_185827.jpg', 'data/obj/g21_Color.jpg', 'data/obj/o_Color.jpeg', 'data/obj/q_Color.jpeg', 'data/obj/17_Color.jpeg', 'data/obj/PXL_20221012_002654592.jpg', 'data/obj/IMG_20221010_185746.jpg', 'data/obj/gnf48_Color.jpeg', 'data/obj/PXL_20221011_013902477.jpg', 'data/obj/s_Color.jpeg', 'data/obj/PXL_20221011_013858519.jpg', 'data/obj/y_Color.jpeg', 'data/obj/gnf51_Color.jpeg', 'data/obj/gnf10_Color.jpeg', 'data/obj/PXL_20

256

In [13]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [14]:
#   only if starting from scratch
#!wget https://pjreddie.com/media/files/darknet53.conv.74
!cp /mydrive/yolov3_gumby_n_friends/yolov3_training_restart0.weights .

In [ ]:
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
!./darknet detector train data/obj.data cfg/yolov3_training.cfg yolov3_training_restart0.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 576074, rewritten_bbox = 0.001562 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.881127), count: 9, class_loss = 0.000606, iou_loss = 0.258319, total_loss = 0.258925 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 576083, rewritten_bbox = 0.001562 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.917394), count: 5, class_loss = 0.000001, iou_loss = 0.024254, total_loss = 0.024254 
v3 (m